# Preprocess the raw survey data

Saves a cleaned version of the survey data to be used in analysis and that can be publicly published (is IRB compliant).

Keeps only data from participants who:

- Have valid Prolific IDs
- Completed the survey
- Passed ALL attention checks

Keeps only the data columns necessary for analysis. 
Drops:
- Prolific ID (privacy risk)
- Unneeded qualtrics metadata

Saves fields metadata to a separate file and drops this metadata from the saved cleaned data.

In [74]:
import numpy as np
import pandas as pd

raw_data_filepath = '../data/online-v1-prolific-sample.csv'
cleaned_data_filepath = '../data/cleaned-online-v1-prolific-sample.csv'
fields_filepath = '../data/online-v1-fields.csv'

In [75]:
df = pd.read_csv(raw_data_filepath)
print(list(df.columns))

['StartDate', 'EndDate', 'Status', 'IPAddress', 'Progress', 'Duration (in seconds)', 'Finished', 'RecordedDate', 'ResponseId', 'RecipientLastName', 'RecipientFirstName', 'RecipientEmail', 'ExternalReference', 'LocationLatitude', 'LocationLongitude', 'DistributionChannel', 'UserLanguage', 'ProlificID ', 'Q_attn_1', 'Q_attn_2', 'Q_sandwich_1', 'Q_sandwich_2', 'Q_sandwich_3', 'Q_sandwich_4', 'Q_sandwich_5', 'Q_sandwich_1_labeled', 'Q_sandwich_2_labeled', 'Q_sandwich_3_labeled', 'Q_sandwich_4_labeled', 'Q_sandwich_5_labeled', 'Q_attn_check', 'Q_current_hunger', 'Q_priorities_1', 'Q_priorities_2', 'Q_priorities_3', 'Q_priorities_4', 'Q_priorities_5', 'Q_special_diet', 'Q_special_diet_6_TEXT', 'Q_past_vegetarian', 'Q_past_vegan', 'Q_vegetarian_not_me', 'Q_vegan_not_me', 'Q_good_for_env', 'Q_how_important_env', 'Q_attn_midway', 'Q_politics', 'Q_US_state', 'Q_live_urban', 'Q_work_urban', 'Q_work_urban_3_TEXT', 'Q_demos_age', 'Q_demos_gender', 'Q_demos_income', 'Q_demos_race', 'Q_comments', 'PR

In [76]:
df.drop(['ProlificID ','PROLIFIC_PID'], axis=1).head()

,StartDate,EndDate,Status,IPAddress,Progress,Duration (in seconds),Finished,RecordedDate,ResponseId,RecipientLastName,...,Q_live_urban,Q_work_urban,Q_work_urban_3_TEXT,Q_demos_age,Q_demos_gender,Q_demos_income,Q_demos_race,Q_comments,STUDY_ID,treatment
0,Start Date,End Date,Response Type,IP Address,Progress,Duration (in seconds),Finished,Recorded Date,Response ID,Recipient Last Name,...,Which best describes the area you live?,Do you work in an urban area? - Selected Choice,Do you work in an urban area? - Not sure / Oth...,How old are you?,What is your gender?,What was your total household income before ta...,Choose one or more races that you consider you...,Any extra comments? (optional),STUDY_ID,treatment
1,"{""ImportId"":""startDate"",""timeZone"":""America/De...","{""ImportId"":""endDate"",""timeZone"":""America/Denv...","{""ImportId"":""status""}","{""ImportId"":""ipAddress""}","{""ImportId"":""progress""}","{""ImportId"":""duration""}","{""ImportId"":""finished""}","{""ImportId"":""recordedDate"",""timeZone"":""America...","{""ImportId"":""_recordId""}","{""ImportId"":""recipientLastName""}",...,"{""ImportId"":""QID60""}","{""ImportId"":""QID63""}","{""ImportId"":""QID63_3_TEXT""}","{""ImportId"":""QID30""}","{""ImportId"":""QID57""}","{""ImportId"":""QID25""}","{""ImportId"":""QID24""}","{""ImportId"":""QID58_TEXT""}","{""ImportId"":""STUDY_ID""}","{""ImportId"":""treatment""}"
2,2022-06-28 09:30:49,2022-06-28 09:32:52,0,*******,100,123,1,2022-06-28 09:32:52,R_125SdSXI0xLI29S,*******,...,2,1,NaN,4,1,6,1,NaN,62bb1ba7f2c710e36fd2c1be,labeled
3,2022-06-28 09:30:31,2022-06-28 09:34:22,0,*******,100,231,1,2022-06-28 09:34:23,R_23V4Eyckqxl6Bmy,*******,...,2,2,NaN,7,1,5,1,I do not lean toward either party. I am a libe...,62bb1ba7f2c710e36fd2c1be,unlabeled
4,2022-06-28 09:31:35,2022-06-28 09:35:34,0,*******,100,239,1,2022-06-28 09:35:35,R_2c5wLF9HxTeZ30h,*******,...,1,2,NaN,5,2,2,1,"None, but thank you!",62bb1ba7f2c710e36fd2c1bf,labeled


Can immediately drop the columns never used that were added by Qualtrics.

In [77]:
drop_cols = [
    'StartDate', 'EndDate', 'Status', 'IPAddress', 'Progress',
    'RecipientLastName', 'RecipientFirstName', 'RecipientEmail', 'ExternalReference', 
    'LocationLatitude', 'LocationLongitude', 'DistributionChannel', 
    'UserLanguage', 
    'ProlificID ', # use PROLIFIC_ID instead which is a URL parameter
]
df = df.drop(drop_cols, axis=1)
df.drop(['PROLIFIC_PID'], axis=1).head()

,Duration (in seconds),Finished,RecordedDate,ResponseId,Q_attn_1,Q_attn_2,Q_sandwich_1,Q_sandwich_2,Q_sandwich_3,Q_sandwich_4,...,Q_live_urban,Q_work_urban,Q_work_urban_3_TEXT,Q_demos_age,Q_demos_gender,Q_demos_income,Q_demos_race,Q_comments,STUDY_ID,treatment
0,Duration (in seconds),Finished,Recorded Date,Response ID,Please enter the number you see here into the ...,Help us keep track of who is paying attention....,Which item would you prefer?,Which item would you prefer?,Which item would you prefer?,Which item would you prefer?,...,Which best describes the area you live?,Do you work in an urban area? - Selected Choice,Do you work in an urban area? - Not sure / Oth...,How old are you?,What is your gender?,What was your total household income before ta...,Choose one or more races that you consider you...,Any extra comments? (optional),STUDY_ID,treatment
1,"{""ImportId"":""duration""}","{""ImportId"":""finished""}","{""ImportId"":""recordedDate"",""timeZone"":""America...","{""ImportId"":""_recordId""}","{""ImportId"":""QID31_TEXT""}","{""ImportId"":""QID32""}","{""ImportId"":""QID40""}","{""ImportId"":""QID41""}","{""ImportId"":""QID42""}","{""ImportId"":""QID43""}",...,"{""ImportId"":""QID60""}","{""ImportId"":""QID63""}","{""ImportId"":""QID63_3_TEXT""}","{""ImportId"":""QID30""}","{""ImportId"":""QID57""}","{""ImportId"":""QID25""}","{""ImportId"":""QID24""}","{""ImportId"":""QID58_TEXT""}","{""ImportId"":""STUDY_ID""}","{""ImportId"":""treatment""}"
2,123,1,2022-06-28 09:32:52,R_125SdSXI0xLI29S,21,5,NaN,NaN,NaN,NaN,...,2,1,NaN,4,1,6,1,NaN,62bb1ba7f2c710e36fd2c1be,labeled
3,231,1,2022-06-28 09:34:23,R_23V4Eyckqxl6Bmy,21,5,0,0,0,1,...,2,2,NaN,7,1,5,1,I do not lean toward either party. I am a libe...,62bb1ba7f2c710e36fd2c1be,unlabeled
4,239,1,2022-06-28 09:35:35,R_2c5wLF9HxTeZ30h,21,5,NaN,NaN,NaN,NaN,...,1,2,NaN,5,2,2,1,"None, but thank you!",62bb1ba7f2c710e36fd2c1bf,labeled


Save the fields metadata in case someone wants it for easy reference later. Then drop from the preprocessed data.

In [78]:
fields_df = df.loc[0].rename('field').to_frame()
fields_df

,field
Duration (in seconds),Duration (in seconds)
Finished,Finished
RecordedDate,Recorded Date
ResponseId,Response ID
Q_attn_1,Please enter the number you see here into the ...
Q_attn_2,Help us keep track of who is paying attention....
Q_sandwich_1,Which item would you prefer?
Q_sandwich_2,Which item would you prefer?
Q_sandwich_3,Which item would you prefer?
Q_sandwich_4,Which item would you prefer?


In [79]:
fields_df.to_csv(fields_filepath)

In [80]:
df = df.drop([0,1])
df.drop(['PROLIFIC_PID'], axis=1).head(3)

,Duration (in seconds),Finished,RecordedDate,ResponseId,Q_attn_1,Q_attn_2,Q_sandwich_1,Q_sandwich_2,Q_sandwich_3,Q_sandwich_4,...,Q_live_urban,Q_work_urban,Q_work_urban_3_TEXT,Q_demos_age,Q_demos_gender,Q_demos_income,Q_demos_race,Q_comments,STUDY_ID,treatment
2,123,1,2022-06-28 09:32:52,R_125SdSXI0xLI29S,21,5,NaN,NaN,NaN,NaN,...,2,1,NaN,4,1,6,1,NaN,62bb1ba7f2c710e36fd2c1be,labeled
3,231,1,2022-06-28 09:34:23,R_23V4Eyckqxl6Bmy,21,5,0,0,0,1,...,2,2,NaN,7,1,5,1,I do not lean toward either party. I am a libe...,62bb1ba7f2c710e36fd2c1be,unlabeled
4,239,1,2022-06-28 09:35:35,R_2c5wLF9HxTeZ30h,21,5,NaN,NaN,NaN,NaN,...,1,2,NaN,5,2,2,1,"None, but thank you!",62bb1ba7f2c710e36fd2c1bf,labeled


Drop participants' data based on exclusion criteria.

In [81]:
print('%s initial responses' % len(df))

804 initial responses


In [82]:
# Only include completed surveys via Prolific
N_PROLIFIC_PID_na = len(df[df['PROLIFIC_PID'].isna()])
print('dropping %s non PROLIFIC responses' % N_PROLIFIC_PID_na)
df = df[df['PROLIFIC_PID'].notna()]
# prolific provides a full URL with a PROLIFIC_ID to test preview the survey.
# A participant with this PID is our test
TEST_PREVIEW_PROLIFIC_PID = '62a2a94d2c2d3cac1cdb636b'
print('dropping %s test responses' % len(df[df['PROLIFIC_PID']==TEST_PREVIEW_PROLIFIC_PID]))
df = df[df['PROLIFIC_PID']!=TEST_PREVIEW_PROLIFIC_PID]
N_unfinished = len(df[df['Finished'] != '1'])
df = df[df['Finished'] == '1']
print('dropping %s incomplete responses' % N_unfinished)
print('%s remaining responses' % len(df))

dropping 1 non PROLIFIC responses
dropping 5 test responses
dropping 6 incomplete responses
792 remaining responses


Can now drop 'Finished' and 'PROLIFIC_PID'

In [83]:
df = df.drop(['PROLIFIC_PID','Finished'], axis=1)
df.head(3)

,Duration (in seconds),RecordedDate,ResponseId,Q_attn_1,Q_attn_2,Q_sandwich_1,Q_sandwich_2,Q_sandwich_3,Q_sandwich_4,Q_sandwich_5,...,Q_live_urban,Q_work_urban,Q_work_urban_3_TEXT,Q_demos_age,Q_demos_gender,Q_demos_income,Q_demos_race,Q_comments,STUDY_ID,treatment
2,123,2022-06-28 09:32:52,R_125SdSXI0xLI29S,21,5,NaN,NaN,NaN,NaN,NaN,...,2,1,NaN,4,1,6,1,NaN,62bb1ba7f2c710e36fd2c1be,labeled
3,231,2022-06-28 09:34:23,R_23V4Eyckqxl6Bmy,21,5,0,0,0,1,0,...,2,2,NaN,7,1,5,1,I do not lean toward either party. I am a libe...,62bb1ba7f2c710e36fd2c1be,unlabeled
4,239,2022-06-28 09:35:35,R_2c5wLF9HxTeZ30h,21,5,NaN,NaN,NaN,NaN,NaN,...,1,2,NaN,5,2,2,1,"None, but thank you!",62bb1ba7f2c710e36fd2c1bf,labeled


In [84]:
def passed_attn_checks(row):
    if not ((row['Q_attn_1'] == 'twenty one') or (float(row['Q_attn_1']) == 21)):
        print(row['ResponseId'], 'failed Q_attn_1')
        return False
    if int(row['Q_attn_2']) != 5:
        print(row['ResponseId'], 'failed Q_attn_2')
        return False
    if row['Q_attn_check'] != '1,2':
        print(row['ResponseId'], 'failed Q_attn_check')
        return False
    if int(row['Q_attn_midway']) != 5:
        print(row['ResponseId'], 'failed Q_attn_midway')
        return False
    return True

In [85]:
df['passed_attn_checks'] = df.apply(passed_attn_checks, axis=1)
passed_attn_checks_ResponseId_list = list(df[df.passed_attn_checks == True].ResponseId)
print('%s/%s respondents passed all attention checks' % (
    len(passed_attn_checks_ResponseId_list), len(df)))
print('(%s respondents failed at least one attention check)' % (
    len(df) - len(passed_attn_checks_ResponseId_list)))

# can drop attn check questions
df = df.drop(['Q_attn_1', 'Q_attn_2', 'Q_attn_check', 'Q_attn_midway'], axis=1)
# print(passed_attn_checks_ResponseId_list)
df.head(3)

R_1Nagt8YYb9KXsgu failed Q_attn_check
R_25u12nxsbq17quY failed Q_attn_2
R_1ewLnYbEh64px5y failed Q_attn_2
R_3EbCFbWypJAH9eU failed Q_attn_2
R_31sGHSw29sejYZu failed Q_attn_check
R_XNtU8UZMHfqK8lX failed Q_attn_2
R_1Ov4PG2EqkoaNBI failed Q_attn_2
R_2sbRJWBoxeF0yHs failed Q_attn_2
R_3Hho2J2KAXLsDob failed Q_attn_2
R_3HUPi75tTdT9pnD failed Q_attn_2
R_1eLSotOLp5DjqJP failed Q_attn_2
R_1LLnNW2RfOKf93z failed Q_attn_midway
R_3fvnmHGH7Bmp938 failed Q_attn_2
R_33mAmfZaIBDBZiT failed Q_attn_2
R_3g8l72aZYNXtcZh failed Q_attn_check
R_1CjxgolIeuRdypP failed Q_attn_check
R_UDAbbOpDeLg8Jot failed Q_attn_check
R_1HqLpaBDw6P827V failed Q_attn_2
R_pR8DHCNXAHa8ZFv failed Q_attn_2
R_ahgvJfr1UphyDap failed Q_attn_check
R_1GVKbaKQiuw141N failed Q_attn_2
R_3lDnbCj1Va514U1 failed Q_attn_midway
R_3CB2nXKtOtqSghu failed Q_attn_check
R_d6luDeloHVBgY8N failed Q_attn_check
R_3JCVW4c4FX5Mfft failed Q_attn_2
R_2uEDdPxdTo2mM2d failed Q_attn_check
R_2ePa8YJwMmK9iGx failed Q_attn_2
R_1CHYyN5FSyw54Pg failed Q_attn_chec

,Duration (in seconds),RecordedDate,ResponseId,Q_sandwich_1,Q_sandwich_2,Q_sandwich_3,Q_sandwich_4,Q_sandwich_5,Q_sandwich_1_labeled,Q_sandwich_2_labeled,...,Q_work_urban,Q_work_urban_3_TEXT,Q_demos_age,Q_demos_gender,Q_demos_income,Q_demos_race,Q_comments,STUDY_ID,treatment,passed_attn_checks
2,123,2022-06-28 09:32:52,R_125SdSXI0xLI29S,NaN,NaN,NaN,NaN,NaN,0,0,...,1,NaN,4,1,6,1,NaN,62bb1ba7f2c710e36fd2c1be,labeled,True
3,231,2022-06-28 09:34:23,R_23V4Eyckqxl6Bmy,0,0,0,1,0,NaN,NaN,...,2,NaN,7,1,5,1,I do not lean toward either party. I am a libe...,62bb1ba7f2c710e36fd2c1be,unlabeled,True
4,239,2022-06-28 09:35:35,R_2c5wLF9HxTeZ30h,NaN,NaN,NaN,NaN,NaN,0,0,...,2,NaN,5,2,2,1,"None, but thank you!",62bb1ba7f2c710e36fd2c1bf,labeled,True


Drop responses from participants who failed ANY attention check

In [86]:
print('dropping responses from participants who failed any attention check')
df = df[df['passed_attn_checks']].drop(['passed_attn_checks'], axis=1)

dropping responses from participants who failed any attention check


How long did the survey take people on average?

(in minutes)
Use Median (50%) because there were outliers that took a while.

In [87]:
df['Duration (in seconds)'].astype(int).describe()/60

count      12.650000
mean        8.169851
std       115.370115
min         1.333333
25%         2.650000
50%         3.316667
75%         4.375000
max      3181.650000
Name: Duration (in seconds), dtype: float64

In [88]:
df.head(3)

,Duration (in seconds),RecordedDate,ResponseId,Q_sandwich_1,Q_sandwich_2,Q_sandwich_3,Q_sandwich_4,Q_sandwich_5,Q_sandwich_1_labeled,Q_sandwich_2_labeled,...,Q_live_urban,Q_work_urban,Q_work_urban_3_TEXT,Q_demos_age,Q_demos_gender,Q_demos_income,Q_demos_race,Q_comments,STUDY_ID,treatment
2,123,2022-06-28 09:32:52,R_125SdSXI0xLI29S,NaN,NaN,NaN,NaN,NaN,0,0,...,2,1,NaN,4,1,6,1,NaN,62bb1ba7f2c710e36fd2c1be,labeled
3,231,2022-06-28 09:34:23,R_23V4Eyckqxl6Bmy,0,0,0,1,0,NaN,NaN,...,2,2,NaN,7,1,5,1,I do not lean toward either party. I am a libe...,62bb1ba7f2c710e36fd2c1be,unlabeled
4,239,2022-06-28 09:35:35,R_2c5wLF9HxTeZ30h,NaN,NaN,NaN,NaN,NaN,0,0,...,1,2,NaN,5,2,2,1,"None, but thank you!",62bb1ba7f2c710e36fd2c1bf,labeled


Save cleaned data to file.

In [89]:
print('saving data to %s...' % cleaned_data_filepath)
df.to_csv(cleaned_data_filepath, index=False)
print('...saved')

saving data to ../data/cleaned-online-v1-prolific-sample.csv...
...saved


In [90]:
# reading from file
clean_df = pd.read_csv(cleaned_data_filepath)
print('%s responses' % len(clean_df))
clean_df.head()

759 responses


,Duration (in seconds),RecordedDate,ResponseId,Q_sandwich_1,Q_sandwich_2,Q_sandwich_3,Q_sandwich_4,Q_sandwich_5,Q_sandwich_1_labeled,Q_sandwich_2_labeled,...,Q_live_urban,Q_work_urban,Q_work_urban_3_TEXT,Q_demos_age,Q_demos_gender,Q_demos_income,Q_demos_race,Q_comments,STUDY_ID,treatment
0,123,2022-06-28 09:32:52,R_125SdSXI0xLI29S,NaN,NaN,NaN,NaN,NaN,0.0,0.0,...,2,1,NaN,4,1,6,1,NaN,62bb1ba7f2c710e36fd2c1be,labeled
1,231,2022-06-28 09:34:23,R_23V4Eyckqxl6Bmy,0.0,0.0,0.0,1.0,0.0,NaN,NaN,...,2,2,NaN,7,1,5,1,I do not lean toward either party. I am a libe...,62bb1ba7f2c710e36fd2c1be,unlabeled
2,239,2022-06-28 09:35:35,R_2c5wLF9HxTeZ30h,NaN,NaN,NaN,NaN,NaN,0.0,0.0,...,1,2,NaN,5,2,2,1,"None, but thank you!",62bb1ba7f2c710e36fd2c1bf,labeled
3,80,2022-06-28 09:36:13,R_2B97XuOwuxSeQWA,0.0,0.0,1.0,1.0,0.0,NaN,NaN,...,2,2,NaN,4,1,2,1,NaN,62bb1ba7f2c710e36fd2c1be,unlabeled
4,134,2022-06-28 09:37:47,R_2BrfiuSgy1SNVbL,NaN,NaN,NaN,NaN,NaN,1.0,1.0,...,3,2,NaN,3,1,4,2,NaN,62bb1ba7f2c710e36fd2c1be,labeled
